In [3]:
%load_ext autoreload
%autoreload 2

from spot.utils import proj_root, os
os.chdir(proj_root())

In [2]:
from spot.type_check import parse_type_str, normalize_type

normalize_type(parse_type_str("int | str"))


ty'Union[int, str]'

In [58]:
bad_code = """
def show_dict_with_change(d: dict, prev: Optional[dict]):
    result = dict()
    for k in d:
        v = d[k]
        v0 = prev.get(k, None) if prev is not None else None
        match v, v0:
            case (CountedAcc() as v, CountedAcc() as v0):
                pass
            case (CountedAcc(), a):
                result[k] = f"{str(v)}"
    return result
"""
from spot.experiments.type4py import cst, remove_newer_syntax

bad_code = remove_newer_syntax(cst.parse_module(bad_code)).code
print(bad_code)



from typing import Any, List, Tuple, Dict, Set, Union, Optional
def show_dict_with_change(d: Dict, prev: Union[Dict, None]):
    result = dict()
    for k in d:
        v = d[k]
        v0 = prev.get(k, None) if prev is not None else None
        if (v, v0) == (CountedAcc(), CountedAcc()):
            pass
        elif (v, v0) == (CountedAcc(), a):
            result[k] = f"{str(v)}"
    return result



In [55]:
import requests
from spot.utils import proj_root, read_file


r = requests.post("https://type4py.com/api/predict?tc=0", bad_code)
r.json()


{'error': None,
 'response': {'classes': [],
  'funcs': [{'docstring': {'func': None, 'long_descr': None, 'ret': None},
    'fn_lc': [[3, 0], [12, 17]],
    'fn_var_ln': {'result': [[4, 4], [4, 10]],
     'v': [[6, 8], [6, 9]],
     'v0': [[7, 8], [7, 10]]},
    'fn_var_occur': {'result': [['result', 'dict'],
      ['result', 'k', 'str', 'v']],
     'v': [['v', 'd', 'k'],
      ['v', 'v0', 'CountedAcc', 'CountedAcc'],
      ['v', 'v0', 'CountedAcc', 'a'],
      ['result', 'k', 'str', 'v']],
     'v0': [['v0', 'prev', 'get', 'k', 'None', 'prev', 'None', 'None'],
      ['v', 'v0', 'CountedAcc', 'CountedAcc'],
      ['v', 'v0', 'CountedAcc', 'a']]},
    'name': 'show_dict_with_change',
    'params': {'d': 'typing.Dict', 'prev': 'typing.Union[typing.Dict, None]'},
    'params_descr': {'d': '', 'prev': ''},
    'params_occur': {'d': [['v', 'd', 'k']],
     'prev': [['v0', 'prev', 'get', 'k', 'None', 'prev', 'None', 'None']]},
    'params_p': {'args': [],
     'd': [['str', 0.235305535097717

In [14]:
import requests
from spot.utils import proj_root, read_file

ex_file = proj_root() / "src/spot/experiments/type4py.py"
r = requests.post("https://type4py.com/api/predict?tc=0", read_file(ex_file))
r.json()


{'error': 'Could not predict types for the given source file!',
 'response': None}

In [3]:
from spot.experiments.type4py import Type4PyResponseParser
from spot.static_analysis import FunctionSignature

pred_signatures = Type4PyResponseParser("spot.model").parse(r.json())
for path, sig in pred_signatures.items():
    if isinstance(sig, FunctionSignature):
        print(path)
        print("\t" + str(sig))


spot.model/dynamic_dataloader
	FuncSig((collate_fn: cst'torch.Tensor', dataset: cst'torch.Tensor', max_tokens: cst'int', shuffle: cst'Optional[bool]') -> cst'str')
spot.model/DecodingArgs.scale_ctx_size
	MethodSig((factor: cst'float') -> cst'List[str]')
spot.model/DecodingArgs.__repr__
	MethodSig(() -> cst'str')
spot.model/DatasetPredResult.accuracies
	MethodSig((common_type_names: cst'str') -> cst'str')
spot.model/DatasetPredResult.group_by_repo
	MethodSig(() -> cst'dict')
spot.model/ModelWrapper.scale_ctx_size
	MethodSig(() -> cst'str')
spot.model/ModelWrapper.decode_row
	MethodSig((n_labels: cst'int', row: cst'str') -> cst'str')
spot.model/ModelWrapper.predict_on_batch
	MethodSig((batch: cst'bool', num_return_sequences: cst'Callable[..., T]') -> cst'str')
spot.model/ModelWrapper.predict
	MethodSig((dataset: cst'numpy.ndarray', num_return_sequences: cst'int', tqdm_args: cst'bytes') -> cst'str')
spot.model/ModelWrapper.save_pretrained
	MethodSig((path: cst'List[str]') -> MISSING)
spot

In [ ]:
from spot.static_analysis import PythonProject
from spot.function_dataset import check_predicted_signatures
from spot.visualization import pretty_print_dict, assert_eq

ex_proj = PythonProject.from_root(
    proj_root(), file_filter=lambda f: f.name == "model.py"
)
label_signatures = {e.path: e.get_signature() for e in ex_proj.all_elems()}

ubiq_names = {"str", "int", "list", "bool", "float"}
accs = check_predicted_signatures(
    {"spot": pred_signatures}, {"spot": label_signatures}, common_type_names=ubiq_names,
)
pretty_print_dict(accs)


partial_acc: 27.91% (count=43)
full_acc: 25.58% (count=43)
full_acc_by_common:
   rare: 25.58% (count=43)
full_acc_by_cat:
   FuncArg: 40.00% (count=15)
   FuncReturn: 11.11% (count=9)
   ClassAtribute: 21.05% (count=19)
full_acc_by_pos:
   range(0, 1): 20.69% (count=29)
   range(1, 2): 22.22% (count=9)
   range(2, 4): 60.00% (count=5)
avg_label_size: 1.0465
avg_pred_size: 1.0465


In [65]:
from spot.static_analysis import PythonProject
from spot.visualization import pretty_print_dict, assert_eq
from spot.experiments.type4py import eval_type4py_on_project

ex_proj = PythonProject.from_root(proj_root(), ignore_dirs={".venv", "data"})
eval_r = eval_type4py_on_project(ex_proj, max_workers=6)


Calling Type4Py: 100%|██████████| 26/26 [00:21<00:00,  1.23it/s]


In [66]:
from spot.function_dataset import check_predicted_signatures

ubiq_names = {"str", "int", "list", "bool", "float"}
accs = check_predicted_signatures(
    eval_r.pred_maps, eval_r.label_maps, ubiq_names, error_on_mismatched_signature=False
)
pretty_print_dict(accs)


partial_acc: 19.26% (count=1.1k)
full_acc: 18.42% (count=1.1k)
full_acc_by_common:
   rare: 4.90% (count=775)
   common: 53.33% (count=300)
full_acc_by_cat:
   FuncArg: 18.29% (count=607)
   FuncReturn: 20.62% (count=257)
   ClassAtribute: 15.71% (count=210)
   GlobalVar: 100.00% (count=1)
full_acc_by_pos:
   range(0, 1): 18.37% (count=539)
   range(1, 2): 19.32% (count=264)
   range(2, 4): 19.60% (count=199)
   range(4, 8): 13.85% (count=65)
   range(8, 16): 0.00% (count=8)
avg_label_size: 1.1823
avg_pred_size: 1.0521
n_label_types: 1110
n_skipped_types: 0
n_missing_types: 35
